# Circuit Composition - Code Laboratory

**Section 3: Create Circuits - Part 2** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

### Composition Methods
| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `qc.compose()` | `compose(other, qubits=None, clbits=None, inplace=False)` | `QuantumCircuit` | Sequential: same qubits, after in time |
| `qc.tensor()` | `tensor(other, inplace=False)` | `QuantumCircuit` | Parallel: different qubits, side-by-side |
| `qc.append()` | `append(instruction, qargs=None, cargs=None)` | `InstructionSet` | Add single gate/operation |

### Circuit Operations
| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `qc.inverse()` | `inverse()` | `QuantumCircuit` | Reverse + adjoint gates |
| `qc.power()` | `power(exponent)` | `QuantumCircuit` | Repeat circuit n times |
| `qc.decompose()` | `decompose(gates_to_decompose=None)` | `QuantumCircuit` | Expand composite gates |
| `qc.to_gate()` | `to_gate(label=None)` | `Gate` | Convert circuit to gate |

### ⚠️ CRITICAL: compose() vs tensor()
| Feature | `compose()` | `tensor()` |
|---------|-------------|------------|
| **Direction** | Time (→) | Space (⊗) |
| **Qubits** | Same | Combined (new) |
| **Width** | Unchanged | Increased |
| **Depth** | Increased | Max of both |

---

In [ ]:
"""
Qiskit Code Laboratory - Circuit Composition
=============================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit.library import QFTGate, XGate, CXGate

# Utility functions for this notebook
def verify_state(qc, expected_state, tolerance=1e-10):
    """Verify circuit produces expected state."""
    actual = Statevector.from_instruction(qc)
    return np.allclose(actual.data, expected_state, atol=tolerance)

def show_state(qc, label=""):
    """Display statevector with optional label."""
    sv = Statevector.from_instruction(qc)
    print(f"{label}: {sv}" if label else f"{sv}")
    return sv

def compare_circuits(qc1, qc2, label1="Circuit 1", label2="Circuit 2"):
    """Compare two circuits' outputs."""
    sv1 = Statevector.from_instruction(qc1)
    sv2 = Statevector.from_instruction(qc2)
    equivalent = np.allclose(sv1.data, sv2.data)
    print(f"{label1}: {sv1}")
    print(f"{label2}: {sv2}")
    print(f"Equivalent: {equivalent}")
    return equivalent

print("✅ Environment ready - Circuit Composition Lab")

---

## `QuantumCircuit.compose()` - Sequential Composition

### Signature
```python
QuantumCircuit.compose(other, qubits=None, clbits=None, inplace=False, copy=True) -> QuantumCircuit
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `other` | `QuantumCircuit` | Yes | - | Circuit to compose |
| `qubits` | `list[int]` | No | `None` | Target qubits (default: same indices) |
| `clbits` | `list[int]` | No | `None` | Target classical bits |
| `inplace` | `bool` | No | `False` | Modify original circuit |

### Returns
`QuantumCircuit` - Composed circuit (or self if inplace=True)

---

### Basic Usage

In [ ]:
# ============================================================
# compose() - BASIC USAGE
# ============================================================

# Create two circuits
qc1 = QuantumCircuit(2)
qc1.h([0, 1])

qc2 = QuantumCircuit(2)
qc2.cx(0, 1)

print("Circuit 1 (preparation):")
print(qc1.draw())
print("\nCircuit 2 (entanglement):")
print(qc2.draw())

# Compose: qc1 THEN qc2
full_circuit = qc1.compose(qc2)
print("\nqc1.compose(qc2) - Sequential:")
print(full_circuit.draw())

# Verify properties
assert full_circuit.num_qubits == 2, "Width should stay 2"
assert full_circuit.depth() == qc1.depth() + qc2.depth(), "Depth should add"
print(f"\n✅ Verified: width={full_circuit.num_qubits}, depth={full_circuit.depth()}")

In [ ]:
# ============================================================
# compose() - PARAMETER VARIATIONS: Qubit Mapping
# ============================================================

qc_large = QuantumCircuit(4)
qc_large.h(range(4))

qc_small = QuantumCircuit(2)
qc_small.cx(0, 1)

print("Large circuit (4 qubits):")
print(qc_large.draw())
print("\nSmall circuit (2 qubits):")
print(qc_small.draw())

# Apply small circuit to specific qubits [1, 2]
result = qc_large.compose(qc_small, qubits=[1, 2])

print("\ncompose(qc_small, qubits=[1, 2]):")
print(result.draw())

# Verify: CX should appear on qubits 1 and 2
ops = result.count_ops()
assert 'cx' in ops, "CX gate should be present"
print(f"\n✅ Small circuit applied to qubits 1 and 2")

---

## `QuantumCircuit.tensor()` - Parallel Composition

### Signature
```python
QuantumCircuit.tensor(other, inplace=False) -> QuantumCircuit
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `other` | `QuantumCircuit` | Yes | - | Circuit to tensor with |
| `inplace` | `bool` | No | `False` | Modify original circuit |

### Returns
`QuantumCircuit` - Tensored circuit (qubits combined)

---

### Basic Usage

In [ ]:
# ============================================================
# tensor() - BASIC USAGE
# ============================================================

system1 = QuantumCircuit(2)
system1.h(0)
system1.cx(0, 1)

system2 = QuantumCircuit(2)
system2.x([0, 1])

print("System 1 (2 qubits):")
print(system1.draw())
print("\nSystem 2 (2 qubits):")
print(system2.draw())

# Tensor product: side-by-side
combined = system1.tensor(system2)
print("\nsystem1.tensor(system2) - Parallel:")
print(combined.draw())

# Verify: qubits should ADD
assert combined.num_qubits == 4, f"Expected 4 qubits (2+2), got {combined.num_qubits}"
print(f"\n✅ Total qubits: {combined.num_qubits} = 2 + 2")

---

### ⚠️ TRAP DEMONSTRATION: compose() vs tensor()

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: compose() vs tensor()
# This is the #1 tested concept in Section 3!
# ============================================================

print("⚠️ TRAP: compose() vs tensor()")
print("="*60)

# Create identical sub-circuit
sub = QuantumCircuit(2)
sub.h(0)
sub.cx(0, 1)

print("Sub-circuit (Bell state prep):")
print(sub.draw())

# Method 1: compose (SEQUENTIAL - same qubits)
qc_compose = QuantumCircuit(2)
qc_compose = qc_compose.compose(sub)
qc_compose = qc_compose.compose(sub)  # Run TWICE on same qubits

print("\n" + "="*60)
print("compose() - SEQUENTIAL (same qubits, time direction):")
print("="*60)
print(qc_compose.draw())
print(f"Qubits: {qc_compose.num_qubits} (unchanged!)")
print(f"Depth: {qc_compose.depth()} (increased!)")

# Method 2: tensor (PARALLEL - different qubits)
sub1 = QuantumCircuit(2)
sub1.h(0)
sub1.cx(0, 1)

sub2 = QuantumCircuit(2)
sub2.h(0)
sub2.cx(0, 1)

qc_tensor = sub1.tensor(sub2)

print("\n" + "="*60)
print("tensor() - PARALLEL (different qubits, space direction):")
print("="*60)
print(qc_tensor.draw())
print(f"Qubits: {qc_tensor.num_qubits} (doubled!)")
print(f"Depth: {qc_tensor.depth()} (max of both)")

print("\n" + "="*60)
print("💡 KEY DIFFERENCE:")
print("   compose(): 2 qubits, depth 4 (time →)")
print("   tensor():  4 qubits, depth 2 (space ⊗)")
print("="*60)

---

## `QuantumCircuit.append()` - Adding Gates

### Signature
```python
QuantumCircuit.append(instruction, qargs=None, cargs=None, copy=True) -> InstructionSet
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `instruction` | `Instruction` | Yes | - | Gate or instruction to add |
| `qargs` | `list[int]` | No | `None` | Target qubits |
| `cargs` | `list[int]` | No | `None` | Target classical bits |

---

### Basic Usage

In [ ]:
# ============================================================
# append() - BASIC USAGE
# ============================================================

qc = QuantumCircuit(3)
qc.h([0, 1, 2])

# Append QFT as a single block
qc.append(QFTGate(3), [0, 1, 2])

print("Circuit with appended QFT:")
print(qc.draw())

# Verify QFT was added
assert 'qft' in qc.count_ops(), "QFT should appear in operations"
print("\n✅ QFT appears as single block (not decomposed)")

In [ ]:
# ============================================================
# append() - CUSTOM GATE
# ============================================================

# Create custom Bell operation
bell_circuit = QuantumCircuit(2, name='Bell')
bell_circuit.h(0)
bell_circuit.cx(0, 1)

# Convert to gate
bell_gate = bell_circuit.to_gate()

# Use in another circuit
qc = QuantumCircuit(4)
qc.x(0)  # Prepare initial state
qc.append(bell_gate, [1, 2])  # Apply Bell to qubits 1, 2
qc.h(3)

print("Circuit with custom Bell gate:")
print(qc.draw())

# Verify Bell gate was added
assert 'Bell' in qc.count_ops(), "Bell gate should appear"
print("\n✅ Custom operations shown as named blocks")

---

## `inverse()` and `power()` - Circuit Operations

### Signatures
```python
QuantumCircuit.inverse() -> QuantumCircuit
QuantumCircuit.power(exponent) -> QuantumCircuit
```

---

### inverse() - Reverse + Adjoint

In [ ]:
# ============================================================
# inverse() - BASIC USAGE
# ============================================================

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.t(0)

print("Original circuit:")
print(qc.draw())

# Get inverse: reverses order AND takes adjoint
qc_inv = qc.inverse()

print("\nInverse circuit:")
print(qc_inv.draw())

# Verify: inverse should reverse order
# Original: H → CX → T
# Inverse:  T† → CX† → H†  (note: CX† = CX, H† = H, but T† = Tdg)
ops_inv = list(qc_inv.count_ops().keys())
print(f"\nOperations in inverse: {ops_inv}")
assert 'tdg' in ops_inv, "T† (Tdg) should appear in inverse"
print("✅ Gates reversed and inverted (T† instead of T)")

In [ ]:
# ============================================================
# inverse() - VERIFICATION: Cancellation
# ============================================================

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Original + Inverse = Identity
qc_full = qc.compose(qc.inverse())

print("Original + Inverse:")
print(qc_full.draw())

# Verify: should produce identity operation
sv = Statevector.from_instruction(qc_full)
expected = np.array([1, 0, 0, 0])  # |00⟩
assert np.allclose(sv.data, expected), "Should return to |00⟩"
print(f"\nResulting state: {sv}")
print("✅ Verified: U · U† = I (returns to |00⟩)")

In [ ]:
# ============================================================
# power() - BASIC USAGE
# ============================================================

qc = QuantumCircuit(1)
qc.s(0)  # S gate (√Z)

print("Original (S gate):")
print(qc.draw())

# S² = Z (exam fact!)
qc_squared = qc.power(2)
print("\npower(2) - S²:")
print(qc_squared.draw())

# Verify: S² should equal Z
op_s2 = Operator(qc_squared)
op_z = Operator(QuantumCircuit(1).compose(QuantumCircuit(1, name='z').compose(QuantumCircuit(1))))

# Better verification: compare matrices
qc_z = QuantumCircuit(1)
qc_z.z(0)
op_z = Operator(qc_z)

print(f"\nS² matrix:\n{op_s2.data}")
print(f"\nZ matrix:\n{op_z.data}")
assert np.allclose(op_s2.data, op_z.data), "S² should equal Z"
print("✅ Verified: S² = Z (exam fact!)")

---

## `decompose()` - View Basic Gates

### Signature
```python
QuantumCircuit.decompose(gates_to_decompose=None, reps=1) -> QuantumCircuit
```

---

In [ ]:
# ============================================================
# decompose() - BASIC USAGE
# ============================================================

qc = QuantumCircuit(3)
qc.append(QFTGate(3), range(3))

print("Before decompose():")
print(qc.draw())
print(f"Operations: {qc.count_ops()}")

# Decompose to basic gates
qc_decomposed = qc.decompose()

print("\nAfter decompose():")
print(qc_decomposed.draw())
print(f"Operations: {qc_decomposed.count_ops()}")

# Verify: QFT should be expanded
assert 'qft' not in qc_decomposed.count_ops(), "QFT should be decomposed"
print("\n✅ QFT expanded into basic gates")

---

## 🏋️ Code Challenges

> These are CODING challenges - solve them by writing code.
> For conceptual questions, see [README.md](./README.md) practice section.

---

In [ ]:
# ============================================================
# CHALLENGE 1: 4-Qubit Bell States
# ============================================================
# Task: Create TWO independent Bell states using tensor()
# Expected: 4 qubits, each pair entangled independently
# ============================================================

def challenge_1():
    """
    Create a 4-qubit circuit with 2 independent Bell states.
    - Qubits 0-1: Bell state
    - Qubits 2-3: Bell state
    
    Use tensor() to combine them.
    
    Returns:
        QuantumCircuit with 4 qubits
    """
    # YOUR CODE HERE
    bell = QuantumCircuit(2)
    # TODO: Create Bell state
    # TODO: Use tensor() to combine two Bell circuits
    
    return bell  # Replace with combined circuit

# Uncomment to test:
# qc = challenge_1()
# print("Your circuit:")
# print(qc.draw())
# print(f"\nQubits: {qc.num_qubits} (expected: 4)")
# if qc.num_qubits == 4:
#     print("✅ Challenge 1 PASSED!")
# else:
#     print("❌ Challenge 1 FAILED")

In [ ]:
# ============================================================
# SOLUTION: Challenge 1
# ============================================================

def challenge_1_solution():
    """Two independent Bell states using tensor()."""
    bell = QuantumCircuit(2)
    bell.h(0)
    bell.cx(0, 1)
    
    # tensor() creates independent systems
    combined = bell.tensor(bell)
    return combined

qc_sol = challenge_1_solution()
print("Solution:")
print(qc_sol.draw())
print(f"\n✅ {qc_sol.num_qubits} qubits with 2 independent Bell states")

---

## `to_gate()` and `to_instruction()` - Reusable Components

### Key Difference (EXAM!)

| Method | Returns | Can use with `control()` | Has params |
|--------|---------|-------------------------|------------|
| `to_gate()` | `Gate` | ✅ Yes | ✅ Preserves |
| `to_instruction()` | `Instruction` | ❌ No | ✅ Preserves |

In [ ]:
# ============================================================
# to_gate() - Convert Circuit to Reusable Gate
# ============================================================

# Create a custom operation
my_circuit = QuantumCircuit(2, name='MyOp')
my_circuit.h(0)
my_circuit.cx(0, 1)
my_circuit.rz(np.pi/4, 1)

# Convert to gate
my_gate = my_circuit.to_gate()
print(f"Gate name: {my_gate.name}")
print(f"Gate qubits: {my_gate.num_qubits}")

# Use in another circuit
qc = QuantumCircuit(4)
qc.append(my_gate, [0, 1])   # First instance
qc.append(my_gate, [2, 3])   # Second instance (reuse!)

print("\nCircuit using custom gate twice:")
print(qc.draw())
print("\n✅ to_gate() allows reusing circuit as single operation")

In [ ]:
# ============================================================
# to_gate().control() - Create Controlled Version (EXAM!)
# ============================================================

# Create simple gate
simple = QuantumCircuit(1, name='U')
simple.h(0)
simple.t(0)

# Convert and add control
simple_gate = simple.to_gate()
controlled_gate = simple_gate.control(1)  # Add 1 control qubit

print(f"Original qubits: {simple_gate.num_qubits}")
print(f"Controlled qubits: {controlled_gate.num_qubits}")

# Use controlled gate
qc = QuantumCircuit(3)
qc.x(0)  # Control qubit = |1⟩
qc.append(controlled_gate, [0, 1])  # Control on q0, target on q1

print("\nControlled custom gate:")
print(qc.draw())
print("\n✅ control(n) adds n control qubits to ANY gate")

In [ ]:
# ============================================================
# ⚠️ TRAP: to_gate() vs to_instruction()
# ============================================================

# Create circuit
qc_sub = QuantumCircuit(2, name='Sub')
qc_sub.h(0)
qc_sub.cx(0, 1)

# to_gate() - CAN be controlled
gate = qc_sub.to_gate()
controlled = gate.control(1)  # ✅ Works!
print(f"✅ gate.control(1) works: {controlled.num_qubits} qubits")

# to_instruction() - CANNOT be controlled
instruction = qc_sub.to_instruction()
try:
    # ❌ This will fail!
    controlled_inst = instruction.control(1)
    print("❌ Should have failed!")
except AttributeError as e:
    print(f"❌ instruction.control() fails: 'Instruction' has no 'control'")

print("\n⚠️ EXAM TRAP:")
print("   to_gate() → Can use .control()")  
print("   to_instruction() → CANNOT use .control()")

---

## 📚 Resources

### Code References
- [Qiskit QuantumCircuit API](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit)
- [Qiskit Circuit Library](https://docs.quantum.ibm.com/api/qiskit/circuit_library)

### Related Materials
- [README.md](./README.md) - Conceptual explanations, traps, mnemonics
- [Practice Questions](./README.md#practice-exam) - Text-based exam practice

### Next Notebook
- [Parameterized Circuits](./parameterized_circuits.ipynb) - Parameters for VQE and QAOA

---

### Quick Reference Card

```
┌─────────────────────────────────────────────────────────────────┐
│ CIRCUIT COMPOSITION - QUICK REFERENCE                           │
├─────────────────────────────────────────────────────────────────┤
│ COMPOSITION:                                                    │
│   qc.compose(other)     - Sequential (same qubits, time →)     │
│   qc.tensor(other)      - Parallel (more qubits, space ⊗)      │
│   qc.append(gate, qargs) - Add single gate                     │
├─────────────────────────────────────────────────────────────────┤
│ OPERATIONS:                                                     │
│   qc.inverse()          - Reverse + adjoint                    │
│   qc.power(n)           - Repeat n times                       │
│   qc.decompose()        - Expand composite gates               │
│   qc.to_gate()          - Convert to reusable gate             │
├─────────────────────────────────────────────────────────────────┤
│ KEY FACTS:                                                      │
│   compose: width unchanged, depth increases                     │
│   tensor:  width increases, depth = max(both)                  │
│   inverse: reverses ORDER + takes ADJOINT (T → T†)             │
│   S² = Z  (power exam fact!)                                   │
└─────────────────────────────────────────────────────────────────┘
```

---

**Notebook verified with Qiskit 1.x** | Last updated: 2025